In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

2022-11-18 22:22:12.860722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 22:22:12.996558: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-18 22:22:13.027812: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 22:22:13.569940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

# Take the sign names in our collections data

In [2]:
signs = sorted(os.listdir("./try_data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [5]:
signs[0]

'hello'

In [6]:
sign_id['hello']

0

# Create the our x and y data


In [7]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [13]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1530)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [14]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [15]:
model.fit(x_train, y_train, epochs=80) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/80
9/9 [==============================] - 13s 87ms/step - loss: 2.1441 - categorical_accuracy: 0.2175
Epoch 2/80
9/9 [==============================] - 1s 76ms/step - loss: 1.7825 - categorical_accuracy: 0.2105
Epoch 3/80
9/9 [==============================] - 1s 79ms/step - loss: 1.8747 - categorical_accuracy: 0.1860
Epoch 4/80
9/9 [==============================] - 1s 128ms/step - loss: 1.7574 - categorical_accuracy: 0.2316
Epoch 5/80
9/9 [==============================] - 4s 410ms/step - loss: 1.6385 - categorical_accuracy: 0.2526
Epoch 6/80
9/9 [==============================] - 4s 413ms/step - loss: 1.5694 - categorical_accuracy: 0.3123
Epoch 7/80
9/9 [==============================] - 4s 413ms/step - loss: 1.4330 - categorical_accuracy: 0.3860
Epoch 8/80
9/9 [==============================] - 4s 399ms/step - loss: 1.2802 - categorical_accuracy: 0.3825
Epoch 9/80
9/9 [==============================] - 4s 398ms/step - loss: 1.5989 - categorical_accuracy: 0.3053
Epoch 10/80


In [9]:
del model

In [30]:
model.summary() # summary of our model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 30, 64)            49408     
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_9 (Dense)             (None, 32)                61472     
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 13)               

In [58]:
res = model.predict(x_test)

1/1 [==============================] - 1s 593ms/step


In [59]:
signs[np.argmax(res[5])]

'thank you'

In [60]:
signs[np.argmax(y_test[5])]

'thank you'

# save weights


In [16]:
model.save("VertoMotus_MLmodel4.h5") # save model

In [89]:
model.save('VertoMotus_MLmodel(2.0- 97).model')

INFO:tensorflow:Assets written to: VertoMotus_MLmodel(2.0- 97).model\assets


In [17]:
model = tf.keras.models.load_model("./VertoMotus_MLmodel4.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./normalized_data.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: /tmp/tmp_zvojqrl/assets


2022-11-18 22:31:39.550677: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-18 22:31:39.551810: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-18 22:31:39.586191: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_zvojqrl
2022-11-18 22:31:39.634664: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-18 22:31:39.659592: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp_zvojqrl
2022-11-18 22:31:39.856277: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-18 22:31:39.886296: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-18 22:31:40.677595: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp_zvojqrl
2022-11

651464

In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 